In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

In [2]:
import warnings

# 경고 무시하기
warnings.filterwarnings("ignore")
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

In [3]:
df = pd.read_csv('./통합데이터/강남구전세데이터_찐마지막.csv', encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87361 entries, 0 to 87360
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AD_Code         87361 non-null  int64  
 1   Admin_District  87361 non-null  object 
 2   Area            87361 non-null  float64
 3   YearBuilt       87361 non-null  int64  
 4   Type            87361 non-null  object 
 5   Floor           87361 non-null  float64
 6   Address         87361 non-null  object 
 7   Cont_Date       87361 non-null  object 
 8   Age             87361 non-null  int64  
 9   Latitude        87361 non-null  float64
 10  Longitude       87361 non-null  float64
 11  Star_Cnt        87361 non-null  int64  
 12  Michelin_Cnt    87361 non-null  int64  
 13  Mc_cnt          87361 non-null  int64  
 14  Bigstore_cnt    87361 non-null  int64  
 15  InterestRate    87361 non-null  float64
 16  Market_cnt      87361 non-null  int64  
 17  Subway_cnt      87361 non-null 

## geopandas를 이용한 행정구역 선그리기

In [43]:
!pip install geopandas

In [10]:
import geopandas as gpd
import pandas as pd
import os

# http://www.gisdeveloper.co.kr/?p=2332
# 위 페이지에서 읍면동 최신파일을 다운받습니다.

# 아래 path는 제 기준 경로입니다.
# 다운받으신 경로를 제대로 넣어주세요!
path = './emd_20230729 (1)'
emd = gpd.read_file(os.path.join(path, 'emd.shp'), encoding='cp949')

emd.crs="EPSG:5179"

emd['center_point'] = emd['geometry'].geometry.centroid # 선 중심
emd['geometry'] = emd['geometry'].to_crs(epsg=4326) # 후 변환
emd['center_point'] = emd['center_point'].to_crs(epsg=4326)
emd['경도'] = emd['center_point'].map(lambda x: x.xy[0][0])
emd['위도'] = emd['center_point'].map(lambda x: x.xy[1][0])

geo_j = emd[emd['EMD_KOR_NM'].isin(df['Admin_District'].unique())].drop(['center_point'], axis=1).to_json()

m = folium.Map(location=[df['Latitude'].mean() , df['Longitude'].mean()], zoom_start=12.5)

folium.GeoJson(
    geo_j,
    name='지역구'
).add_to(m)

m

# 히트맵

In [7]:
import folium
from folium import plugins
from folium.plugins import HeatMap

m2 = folium.Map(location=[df['Latitude'].mean() , df['Longitude'].mean()], zoom_start=15)

# 분석하고자 하는 데이터프레임을 만드는 코드입니다.
heat_df = df[df['y'] >= df['y'].quantile(0.8)][['Latitude', 'Longitude']]

# 행정구역을 나누는 선을 그리는 코드입니다.
# 위 Cell에 있는 코드입니다. (위 Cell을 먼저 실행시키지 않으면 에러납니다!)
folium.GeoJson(
    geo_j,
    name='지역구'
).add_to(m2)

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data,
       radius=15).add_to(m2)

# Display the map
m2

In [43]:
!pip install geopandas

In [10]:
import geopandas as gpd
import pandas as pd
import os

path = './emd_20230729 (1)'
emd = gpd.read_file(os.path.join(path, 'emd.shp'), encoding='cp949')

emd.crs="EPSG:5179"

emd['center_point'] = emd['geometry'].geometry.centroid # 선 중심
emd['geometry'] = emd['geometry'].to_crs(epsg=4326) # 후 변환
emd['center_point'] = emd['center_point'].to_crs(epsg=4326)
emd['경도'] = emd['center_point'].map(lambda x: x.xy[0][0])
emd['위도'] = emd['center_point'].map(lambda x: x.xy[1][0])

geo_j = emd[emd['EMD_KOR_NM'].isin(df['Admin_District'].unique())].drop(['center_point'], axis=1).to_json()

m = folium.Map(location=[df['Latitude'].mean() , df['Longitude'].mean()], zoom_start=12.5)

folium.GeoJson(
    geo_j,
    name='지역구'
).add_to(m)

m